## Voice Activity Detector (VAD) in Python

In [1]:
import torch
import pyaudio
import numpy as np
from IPython.display import Audio
from pprint import pprint

c:\Users\Hisku\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the JIT file
jit_model = torch.jit.load("./assets/vad_model/vad.jit")

In [3]:
# use pyaudio to get audio stream
p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=8000)

while True:
    data = stream.read(8000)

    # Convert audio data from bytes to numpy array
    data = np.frombuffer(data, dtype=np.float32)

    # Convert numpy array to a tensor
    data_tensor = torch.from_numpy(data)
    data_tensor = torch.reshape(data_tensor, (1, 4000))
    data_tensor = torch.nan_to_num(data_tensor)
    print(data_tensor)
    
    # Send data to AI model
    output = jit_model(data_tensor)

    print(output)

    break

stream.stop_stream()
stream.close()
p.terminate()

C:\Users\Hisku\AppData\Local\Temp\ipykernel_10460\4115080836.py:17: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:205.)
  data_tensor = torch.from_numpy(data)


tensor([[ 8.8024e-35,  3.2214e-36,  6.1718e-37,  ..., -5.5293e+35,
         -3.9456e+35, -3.9456e+35]])


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/models/number_vad_model.py", line 17, in forward
    x0 = (_0)[0]
    x1 = (self.adaptive_normalization).forward(x0, )
    x2 = (self.encoder).forward(x1, )
          ~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    _1 = self.sigmoid
    _2 = torch.squeeze((self.decoder).forward(x2, ), -1)
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_20.py", line 39, in forward
    _14 = getattr(self, "14")
    input0 = (_0).forward(input, )
    input1 = (_1).forward(input0, )
              ~~~~~~~~~~~ <--- HERE
    input2 = (_2).forward(input1, )
    input3 = (_3).forward(input2, )
  File "code/__torch__/models/transformer_modules.py", line 23, in forward
    else:
      x0 = x
    attn = (self.attention).forward(x0, )
            ~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE
    x2 = torch.add(x0, (self.dropout1).forward(attn, ), alpha=1)
 

In [16]:
# use pyaudio to get audio stream
torch.set_num_threads(1)

speech_probs = []
window_size_samples = 512 # use 256 for 8000 Hz model
SAMPLING_RATE = 16000

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_timestamps,
 _, read_audio, VADIterator,
 *_) = utils

vad_iterator = VADIterator(model)


p = pyaudio.PyAudio()

stream = p.open(format=pyaudio.paInt16,
                channels=1,
                rate=16000,
                input=True,
                frames_per_buffer=SAMPLING_RATE)


while True:
    data = stream.read(SAMPLING_RATE)
    data = np.frombuffer(data, dtype=np.float32)
    data_tensor = torch.from_numpy(data)

    chunk = data_tensor
    if len(chunk) < window_size_samples:
      break
    speech_dict = vad_iterator(chunk, return_seconds=True)
    if speech_dict:
        print(speech_dict, end=' ')

    # speech_prob = model(chunk, SAMPLING_RATE).item()
    # speech_probs.append(speech_prob)

stream.stop_stream()
stream.close()
vad_iterator.reset_states()
p.terminate()

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to C:\Users\Hisku/.cache\torch\hub\master.zip


KeyboardInterrupt: 

In [14]:
print(speech_probs)

[]
